In [1]:
%pip install beautifulsoup4 lxml


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pathlib import Path

ASBUILT_DIR = Path() / 'asbuilt'

if not ASBUILT_DIR.is_dir():
  ASBUILT_DIR.mkdir()

def get_asbuilt_path(vin: str) -> Path:
  return ASBUILT_DIR / f'{vin}.ab'

def check_asbuilt_exists(vin: str) -> bool:
  return get_asbuilt_path(vin).is_file()

In [3]:
import pandas as pd

df_vins = pd.read_csv('ford_vins.csv', dtype={"pr":"str"})

duplicates = df_vins[df_vins.duplicated(subset=['vin'], keep=False)]
print(f'Found {len(duplicates)} duplicate VINs')
if len(duplicates):
  print(duplicates)

# remove rows with non-empty "pr" column
df_vins = df_vins[df_vins['pr'].isnull()]
df_vins.drop(columns=['pr'], inplace=True)

# reset index
df_vins.reset_index(drop=True, inplace=True)

print(f"Loaded {len(df_vins)} VINs")
vins = set(df_vins['vin'])

/tmp/ipykernel_37766/335545698.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


FileNotFoundError: [Errno 2] No such file or directory: 'ford_vins.csv'

In [ ]:
print("Download from https://www.motorcraftservice.com/AsBuilt")
missing, found = 0, 0
for vin in df_vins['vin']:
  if not check_asbuilt_exists(vin):
    print(f'missing: {vin}')
    missing += 1
  else:
    found += 1
print(f'missing: {missing}')
print(f'found: {found}')

Download from https://www.motorcraftservice.com/AsBuilt
missing: 0
found: 44


In [ ]:
from bs4 import BeautifulSoup

EcuData = dict[int, str]

def get_car_ecu_data(vin: str) -> dict[int, EcuData]:
  with open(get_asbuilt_path(vin), 'r') as f:
    soup = BeautifulSoup(f, 'lxml')

  car_data = {}
  for value in soup.find_all('nodeid'):
    children = value.children
    addr = int(str(next(children)).strip(), 16)
    ecu_data = {}
    for child in children:
      if child.name:
        data_identifier = int(child.name, 16)
        ecu_data[data_identifier] = child.text.strip()
    car_data[addr] = ecu_data
  return car_data

candidate = df_vins['vin'][0]
print(get_car_ecu_data(candidate))

{1776: {61712: 'DSN1MT-14D068-AA', 61713: 'N1MT-14G635-AA', 61715: 'N1MT-14D068-CA', 61732: 'N1MT-14G631-AA', 61832: 'N1MT-14G630-AA', 61836: '0120360937440391'}, 1795: {61713: 'MB5A-14G581-AA', 61715: 'N1MA-7P238-AA', 61832: 'N1MA-14G576-DA', 61836: '00IFH3Y74801'}, 1798: {61712: 'DSLB5T-19H406-AE', 61713: 'LB5T-14F403-CA', 61715: 'LB5T-19H406-CF', 61728: 'LB5T-14F397-BC', 61732: 'LB5T-14F398-AF', 61733: 'LB5T-14F398-BC', 61832: 'LB5T-14F397-AF', 61836: '220342335'}, 1814: {61706: 'LB5T-14F535-BB', 61712: 'DSJU5T-14F642-BA', 61713: 'LU5T-14F536-EB', 61715: 'LB5T-14F642-BB', 61832: 'LB5T-14F530-BB', 61836: '4710210938323003'}, 1824: {61712: 'DSMB5T-1A292-BA', 61713: 'MB5T-14F094-BA', 61715: 'MB5T-10849-FA', 61732: 'MB5T-14C088-BA', 61832: 'MB5T-14C026-BA', 61836: '03121849'}, 1828: {61712: 'DSLB5T-3F944-AA', 61713: 'LB5T-14F078-DC', 61715: 'LB5T-3F944-SG', 61832: 'LB5T-14C579-AB', 61836: '9291820170497000'}, 1830: {61706: 'NB5T-14C636-AA', 61710: 'LB5T-14G570-AAC', 61712: 'DSLU5T-14B47

In [ ]:
from dataclasses import dataclass
from enum import IntEnum
from typing import NamedTuple

class FordEcu(IntEnum):
  PowerSteeringControlModule = 0x730
  AntiLockBrakeSystem = 0x760
  CruiseControlModule = 0x764
  ImageProcessingModuleA = 0x706
  PowertrainControlModule = 0x7E0
  InstrumentPanelCluster = 0x720
  BodyControlModule = 0x726
  AccessoryProtocolInterfaceModule = 0x7D0
  SteeringColumnControlModule = 0x724

def get_ford_ecu(addr: int) -> FordEcu | None:
  return FordEcu(addr) if addr in FordEcu.__members__.values() else None

@dataclass
class FordFeature:
  masks: list[dict[FordEcu, dict[str, list[int]]]]


class VehicleSetting(NamedTuple):
  address: str
  bit_positions: int
  value_map: dict[int, str]


class VehicleConfiguration:
  configuration_data: dict[str, list[str]]

  def __init__(self, configuration_data: dict[str, list[str]]):
    self.configuration_data = configuration_data

  def get_setting(self, setting: VehicleSetting) -> str:
    codes = self.configuration_data.get(setting.address)
    if not codes:
      return "Not Found"


class FordFeatures:
  # https://www.fordescape.org/threads/how-to-enable-lane-centering-assist.117546/post-1158933
  LaneCenteringAssist = FordFeature(
    masks={
      FordEcu.PowerSteeringControlModule: {
        '02-02': [0x0000, 0xFFFF, 0x00],
      },
      FordEcu.ImageProcessingModuleA: {
        '01-02': [0x0006, 0x0000, 0x00],  # Traffic Jam Assist
        '02-07': [0x0000, 0x0F80, 0x00],
      },
      FordEcu.InstrumentPanelCluster: {
        '10-01': [0x0D00, 0x0000, 0x00],
      },
      FordEcu.SteeringColumnControlModule: {
        # Basic Cruise = 0x0358
        # Adaptive Cruise = 0x0360
        # Adaptive Cruise + Lane Centering = 0x0390
        # TODO: not clear what correct bitmask is
        '02-01': [0x0300, 0x0000, 0x00],
      },
    },
  )
  # https://www.fordescape.org/threads/how-to-adding-adaptive-cruise-non-stop-go-and-copilot-assist.117530/
  AdaptiveCruiseControl = FordFeature(
    masks={
      # FordEcu.PowerSteeringControlModule: {
      #   '02-03': [0xFF00, 0x0000, 0x00],  # enable ESA
      # },
      FordEcu.AntiLockBrakeSystem: {
        # '02-03': [0x0300, 0x00FE, 0x00],  # not needed?
        '03-02': [0x0300, 0x0000, 0x00],  # 0x0400 may also work
        # '03-03': [0x0000, 0x0000, 0x01],  # not needed?
        # '03-04': [0x0000, 0x0000, 0x02],  # not needed?
      },
      FordEcu.CruiseControlModule: {
        # V = VIN as hex
        # 01-01 26VV-VVVV-VV
        # 01-02 VVVV-VVVV-VV
        # 01-03 VVVV-VVVV-VV
        # 01-04 VVVV-Vvxx-xx
        # 01-05 0000-00
        '01-01': [0x2600, 0x0000, 0x00],
      },
      FordEcu.ImageProcessingModuleA: {
        '01-01': [0x0000, 0x9000, 0xA9],
        # 3F = gap setting #1 (closest), 5F = #2, 7F = #3, 9F = #4, BF = #5 (furthest)
        '02-01': [0x0000, 0x0000, 0x1F],
      },
      FordEcu.InstumentPanelCluster: {
        '01-01': [0x0F00, 0x0000, 0x00],  # enable ACC
        # '01-02': [0x0000, 0x80],  # FCW + FDA
        # '09-01': [0x3000, 0x0000, 0x00],  # enable ESA
      },
      FordEcu.BodyControlModule: {
        '04-06': [0x8C00, 0x0000, 0x00],  # 0x8C00 non-stop and go
      },
      FordEcu.AccessoryProtocolInterfaceModule: {
        '09-01': [0x6000, 0x0000, 0x00],  # ACC Menu
        # '09-01': [0x0000, 0x4000, 0x00],  # FCW + FDA
        '09-02': [0x0000, 0x0E00, 0x00],  # enable ACC (0x0F00 with iACC and TSR)
        # '09-02': [0x000C, 0x0000, 0x00],  # enable ESA
      },
      FordEcu.SteeringColumnControlModule: {
        # Basic Cruise = 0x0358
        # Adaptive Cruise = 0x0360
        # Adaptive Cruise + Lane Centering = 0x0390
        # TODO: not clear what correct bitmask is
        '02-01': [0x0390, 0x0000, 0x00],
      },
    },
  )

INTERESTING_ECUS = [ecu for feature in FordFeatures.__dict__.values() if isinstance(feature, FordFeature) for ecu in feature.masks.keys()]

In [ ]:
from collections import defaultdict

def get_module_configuration(vin: str, filter_ecu: FordEcu | None = None) -> dict[FordEcu, dict[str, str]] | dict[str, str]:
  with open(get_asbuilt_path(vin), 'r') as f:
    soup = BeautifulSoup(f, 'lxml')

  # <AS_BUILT_DATA>
  #   <VEHICLE>
  #     <VIN>1FM5K8D8XHGC96884</VIN>
  #     <VEHICLE_DATA>
  #       <DATA LABEL=""><CODE>52E4</CODE><CODE>FFFF</CODE><CODE>FF33</CODE></DATA>
  #     </VEHICLE_DATA>
  #     <PCM_MODULE>
  #       <DATA LABEL="PCM 1"><CODE>FFFF</CODE><CODE>FFFF</CODE><CODE>FF0C</CODE></DATA>
  #       <DATA LABEL="PCM 2"><CODE>FFFF</CODE><CODE>FFFF</CODE><CODE>FF0D</CODE></DATA>
  #       ...
  #     </PCM_MODULE>
  #     <BCE_MODULE>
  #       <DATA LABEL="7D0-01-01"><CODE>2A2A</CODE><CODE>0502</CODE><CODE>083C</CODE></DATA>
  #       <DATA LABEL="7D0-01-02"><CODE>0289</CODE><CODE>0004</CODE><CODE>9A03</CODE></DATA>
  #       ...
  #     </BCE_MODULE>
  #     ...

  module_configuration = defaultdict(dict)
  for module in soup.find_all('bce_module'):
    for data in module.find_all('data'):
      addr, _, label = data['label'].partition('-')

      addr = int(addr, 16)
      ecu = get_ford_ecu(addr)
      if ecu not in INTERESTING_ECUS:
        continue

      if filter_ecu and ecu != filter_ecu:
        continue

      module_configuration[ecu][label] = [code.text for code in data.find_all('code')]

  if filter_ecu:
    return dict(module_configuration[filter_ecu])

  return dict(module_configuration)

for module, conf in get_module_configuration(df_vins['vin'][0]).items():
  print(module.name, conf)

In [ ]:
def get_feature(vin: str, feature: FordFeature) -> bool:
  if ecu not in FORD_FEATURE_MASKS:
    raise ValueError(f'Unknown ECU: {ecu}')
  if feature not in FORD_FEATURE_MASKS[ecu]:
    raise ValueError(f'Unknown feature: {feature}')
  addr, mask = FORD_FEATURE_MASKS[ecu][feature]

  conf = get_module_configuration(vin, ecu)
  if addr not in conf:
    print(f'Unknown address: {addr}')
    return False
  data = conf[addr]
  masked_data = [int(d, 16) & m for d, m in zip(data, mask, strict=True)]
  return all(d == m for d, m in zip(masked_data, mask, strict=True))

for vin in df_vins['vin']:
  print(vin)
  try:
    print(get_feature(vin, FordEcu.ImageProcessingModuleA, 'TrafficJamAssist'))
    print(get_feature(vin, FordEcu.ImageProcessingModuleA, 'LaneCenteringAssist'))
    print(get_feature(vin, FordEcu.PowerSteeringControlModule, 'LaneCenteringAssist'))
  except:
    pass

In [ ]:
import json

NHTSA_DIR = Path() / 'nhtsa'

if not NHTSA_DIR.is_dir():
  NHTSA_DIR.mkdir()

def get_nhtsa_path(vin: str) -> Path:
  return NHTSA_DIR / f'{vin}.json'

def decode_nhtsa_vin_values(vin: str) -> dict[str, str] | None:
  if get_nhtsa_path(vin).is_file():
    with open(get_nhtsa_path(vin), 'r') as f:
      return json.load(f)
  import requests
  url = f'https://vpic.nhtsa.dot.gov/api/vehicles/DecodeVinValuesExtended/{vin}?format=json'
  resp = requests.get(url)
  data = resp.json()['Results'][0]
  if 'ErrorCode' in data and data['ErrorCode'] != '0':
    return None
  with open(get_nhtsa_path(vin), 'w') as f:
    json.dump(data, f)
  return data

In [ ]:
from collections import defaultdict

# {("details", "vin"): [...],
#  ("details", "comment"): [...],
#  ("values", "Make"): [...],
#  ("values", "Model"): [...],
#  ...}
vins_data = defaultdict(list)
known_values = set()

# skip not interesting values
skip_values = {"VIN", "ErrorCode", "ErrorText", "AirBagLocFront", "AirBagLocKnee", "AirBagLocSide",
  "AutoReverseSystem", "AutomaticPedestrianAlertingSound", "Axles", "BasePrice", "CAN_AACN", "CIB",
  "DaytimeRunningLight", "DisplacementCC", "DisplacementCI", "Doors", "ForwardCollisionWarning",
  "MakeID", "ManufacturerId", "ModelID", "NCSAMake", "TPMS", "TractionControl",
  "RearVisibilitySystem", "SeatRows", "Seats", "SemiautomaticHeadlampBeamSwitching", "Wheels"}

for vin, comment in zip(df_vins['vin'], df_vins['comment'], strict=True):
  vin_values = decode_nhtsa_vin_values(vin)
  if vin_values is None:
    print(f'Failed to get NHTSA data for {vin}')
    continue

  vins_data[('details', 'vin')].append(vin)
  vins_data[('details', 'comment')].append(comment)

  found_values = set(vin_values.keys())
  for key in known_values:
    if key not in found_values:
      print('! missing value:', key)
      print('data may be innacurate due to row misalignment')

  for key, value in vin_values.items():
    if key in skip_values:
      continue
    known_values.add(key)
    vins_data[('vin values', key)].append(value)


df_vin_values = pd.DataFrame(vins_data)
print(df_vin_values.shape)

# replace "Not Applicable" with empty string
df_vin_values = df_vin_values.replace("Not Applicable", "")

# remove empty columns
df_vin_values = df_vin_values.loc[:, (df_vin_values != '').any(axis=0)]
print(df_vin_values.shape)

# remove columns with only one value
df_vin_values = df_vin_values.loc[:, df_vin_values.nunique() > 1]
print(df_vin_values.shape)

print(df_vin_values.to_string())

(44, 124)
(44, 52)
(44, 36)
              details                                            vin values                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
                  vin                         comment AdaptiveCruiseControl AdaptiveDrivingBeam BlindSpotMon D

In [ ]:
# statistics
print(df_vin_values['vin values', 'Model'].value_counts())

KeyError: ('vin values', 'Model')

# Compare original openpilot FW matching vs custom fuzzy matcher

In [ ]:
from cereal import car
from panda.python.uds import DATA_IDENTIFIER_TYPE
from openpilot.selfdrive.car.fw_query_definitions import EcuAddrSubAddr
from openpilot.selfdrive.car.ford.tests.test_ford import ECU_ADDRESSES as FW_ECUS

Ecu = car.CarParams.Ecu

FW_ECUS = FW_ECUS.copy()
FW_ECUS.pop(Ecu.shiftByWire)

# ECU_ADDRESS_TO_NAME = {v: k for k, v in ECU_ADDRESSES.items()}

def get_fw_dict(vin: str) -> dict[EcuAddrSubAddr, list[bytes]]:
  car_fw = {}
  for addr, ecu_data in get_car_ecu_data(vin).items():
    if addr not in FW_ECUS.values():
      continue
    # ecu = ECU_ADDRESS_TO_NAME[addr]

    fw = ecu_data[DATA_IDENTIFIER_TYPE.VEHICLE_MANUFACTURER_ECU_SOFTWARE_NUMBER]
    fw_length = 24
    fw = fw.encode()
    fw = (fw + b'\x00' * fw_length)[:fw_length]

    car_fw[(addr, None)] = [fw]
  return car_fw

print(candidate, get_fw_dict(candidate))

1FMSK8DH1NGA52944 {(1798, None): [b'LB5T-14F397-AF\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'], (1840, None): [b'M1MC-14D003-AB\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'], (1888, None): [b'L1MC-2D053-KB\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'], (2016, None): [b'NB5A-14C204-ARC\x00\x00\x00\x00\x00\x00\x00\x00\x00']}


In [ ]:
from cereal import car
from openpilot.selfdrive.car.fw_query_definitions import LiveFwVersions
from openpilot.selfdrive.car.fw_versions import match_fw_to_car_fuzzy as match_op_fuzzy
from openpilot.selfdrive.car.ford.fingerprints import FW_VERSIONS as FORD_FW_VERSIONS
from openpilot.selfdrive.car.ford.values import match_fw_to_car_fuzzy as match_custom_fuzzy

Ecu = car.CarParams.Ecu
ESSENTIAL_ECUS = [Ecu.engine, Ecu.eps, Ecu.abs, Ecu.fwdRadar, Ecu.fwdCamera, Ecu.vsa]

def match_fw_to_car_exact(live_fw_versions: LiveFwVersions) -> set[str]:
  """Do an exact FW match. Returns all cars that match the given
  FW versions for a list of "essential" ECUs. If an ECU is not considered
  essential the FW version can be missing to get a fingerprint, but if it's present it
  needs to match the database."""
  invalid = set()
  candidates = FORD_FW_VERSIONS

  for candidate, fws in candidates.items():
    for ecu, expected_versions in fws.items():
      ecu_type = ecu[0]
      addr = ecu[1:]

      found_versions = live_fw_versions.get(addr, set())
      if not len(found_versions):
        # Some models can sometimes miss an ecu, or show on two different addresses
        # FIXME: this logic can be improved to be more specific, should require one of the two addresses
        # if candidate in config.non_essential_ecus.get(ecu_type, []):
        #   continue

        # Ignore non essential ecus
        if ecu_type not in ESSENTIAL_ECUS:
          continue

      # Virtual debug ecu doesn't need to match the database
      if ecu_type == Ecu.debug:
        continue

      if not any(found_version in expected_versions for found_version in found_versions):
        invalid.add(candidate)
        break

  return set(candidates.keys()) - invalid

def match_fw_to_car(fw_versions_dict: LiveFwVersions, allow_fuzzy: bool, allow_custom_fuzzy: bool) -> tuple[bool, set[str]]:
  # Attempt to fingerprint using all FW returned from its queries
  matches = match_fw_to_car_exact(fw_versions_dict)
  if len(matches):
    return True, matches

  matches = match_op_fuzzy(fw_versions_dict, log=False)

  # If specified and no matches so far, fall back to brand's fuzzy fingerprinting function
  if allow_custom_fuzzy and not len(matches):
    matches |= match_custom_fuzzy(fw_versions_dict, FORD_FW_VERSIONS)

  if len(matches):
    return False, matches

  return True, set()


def get_fingerprint(fw_versions_dict: LiveFwVersions, allow_fuzzy=True, allow_custom_fuzzy=False) -> tuple[bool, str]:
  exact_match, matches = match_fw_to_car(fw_versions_dict, allow_fuzzy, allow_custom_fuzzy)
  if len(matches) == 0:
    return False, "mock"
  elif len(matches) == 1:
    return not exact_match, matches.pop()
  else:
    return not exact_match, "multiple"


def run_fw_test(vins):
  df_matches_rows = []
  columns = ['VIN', 'Model', 'ModelYear', 'ExistingFingerprint', 'ExistingFuzzy', 'NewFingerprint', 'NewFuzzy']
  for vin in vins:
    fw_versions_dict = get_fw_dict(vin)

    # uses exact/fuzzy matching
    fuzzy, fingerprint = get_fingerprint(fw_versions_dict)

    # uses exact/fuzzy matching + custom match_fw_to_car_fuzzy from fw config
    new_fuzzy, new_fingerprint = get_fingerprint(fw_versions_dict, allow_custom_fuzzy=True)

    values = df_vin_values[df_vin_values['details', 'vin'] == vin].iloc[0]
    df_matches_rows.append((vin, 'Explorer', values['vin values', 'ModelYear'],
                            fingerprint, fuzzy, new_fingerprint, new_fuzzy))
  df_matches = pd.DataFrame(df_matches_rows, columns=columns)
  df_matches.sort_values(by=['Model', 'ModelYear'], inplace=True, ascending=[True, True])
  return df_matches

In [ ]:
vins = set(df_vin_values['details', 'vin'])
df_fw_test = run_fw_test(vins)

def get_result(row):
  if row['ExistingFingerprint'] == 'mock' and row['NewFingerprint'] == 'mock':
    return 'mock -> mock'
  elif row['ExistingFingerprint'] == 'mock':
    return 'mock -> fp'
  elif row['NewFingerprint'] == 'mock':
    return 'fp -> mock'
  else:
    return 'fp -> fp'

def trim_vin(row):
  return row['VIN'][0:12] + 'X' * 6

df_fw_test['Result'] = df_fw_test.apply(get_result, axis=1)
df_fw_test['VIN'] = df_fw_test.apply(trim_vin, axis=1)
print(df_fw_test.shape)
df_fw_test

(44, 8)


,VIN,Model,ModelYear,ExistingFingerprint,ExistingFuzzy,NewFingerprint,NewFuzzy,Result
14,1FM5K8GT6HGDXXXXXX,Explorer,2017,mock,False,mock,False,mock -> mock
23,1FM5K8D8XHGCXXXXXX,Explorer,2017,mock,False,mock,False,mock -> mock
29,1FM5K8D83KGAXXXXXX,Explorer,2019,mock,False,mock,False,mock -> mock
0,1FMSK8DH1LGCXXXXXX,Explorer,2020,mock,False,mock,False,mock -> mock
4,1FM5K8GC3LGAXXXXXX,Explorer,2020,FORD EXPLORER 6TH GEN,False,FORD EXPLORER 6TH GEN,False,fp -> fp
5,1FMSK8DH0LGBXXXXXX,Explorer,2020,mock,False,mock,False,mock -> mock
17,1FMSK8DH1LGBXXXXXX,Explorer,2020,FORD EXPLORER 6TH GEN,True,FORD EXPLORER 6TH GEN,True,fp -> fp
20,1FMSK8DH9LGCXXXXXX,Explorer,2020,mock,False,mock,False,mock -> mock
21,1FMSK8DH1LGDXXXXXX,Explorer,2020,mock,False,FORD EXPLORER 6TH GEN,True,mock -> fp
25,1FMSK8DHXLGAXXXXXX,Explorer,2020,mock,False,FORD EXPLORER 6TH GEN,True,mock -> fp


In [ ]:
# statistics
df_fw_test_results = df_fw_test[['Result']].copy()
df_fw_test_results['Count'] = 1
df_fw_test_results = df_fw_test_results.groupby(['Result']).count()
df_fw_test_results.reset_index(inplace=True)
df_fw_test_results.columns = ['Result', 'Count']
df_fw_test_results['%'] = (df_fw_test_results['Count'] / df_fw_test_results['Count'].sum() * 100).round(1)
df_fw_test_results

,Result,Count,%
0,fp -> fp,8,18.2
1,mock -> fp,15,34.1
2,mock -> mock,21,47.7


In [ ]:
from openpilot.selfdrive.car.ford.values import CAR

ALL_ECUS = {k: v for k, v in Ecu.__dict__.items() if isinstance(v, int)}
ECU_LOOKUP = {addr: ecu for ecu, addr in ALL_ECUS.items()}
FW_ECUS_BY_ADDR = {addr: ecu for ecu, addr in FW_ECUS.items()}

def check_fw_database(df):
  for _, row in df.iterrows():
    vin = row['VIN']
    print(row[['VIN', 'Model', 'ModelYear']].to_string())
    print('FW:')

    fw_rows = []
    for addr, fws in get_fw_dict(vin).items():
      ecu = FW_ECUS_BY_ADDR[addr[0]]
      fw = fws[0]

      known_fw = fw in FORD_FW_VERSIONS[CAR.EXPLORER_MK6][(ecu, *addr)]

      fw_rows.append((ECU_LOOKUP[ecu], fw, known_fw))
    df_fw = pd.DataFrame(fw_rows, columns=['ECU', 'FW', 'In Database'])
    print(df_fw.to_string())
    print()

In [ ]:
def is_changed_from_mock(row):
  return row['ExistingFingerprint'] == 'mock' and row['Changed']

df_new_fps = df_fw_test[df_fw_test.apply(is_changed_from_mock, axis=1)].copy()
check_fw_database(df_new_fps)

VIN          1FMSK8DH1LGD04611
Model                 Explorer
ModelYear                 2020
FW:
         ECU                                                            FW  In Database
0  fwdCamera     b'LB5T-14F397-AF\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'         True
1        eps     b'L1MC-14D003-AL\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'         True
2        abs  b'L1MC-2D053-BD\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'         True
3   fwdRadar     b'LB5T-14D049-AB\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'         True
4     engine        b'LB5A-14C204-EAE\x00\x00\x00\x00\x00\x00\x00\x00\x00'        False

VIN          1FMSK8DHXLGA77872
Model                 Explorer
ModelYear                 2020
FW:
         ECU                                                            FW  In Database
0  fwdCamera     b'LB5T-14F397-AE\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'         True
1        eps     b'L1MC-14D003-AJ\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'         True
2        abs 

In [ ]:
def is_not_changed_from_mock(row):
  return row['ExistingFingerprint'] == 'mock' and not row['Changed']

df_mock_unchanged = df_fw_test[df_fw_test.apply(is_not_changed_from_mock, axis=1)].copy()
check_fw_database(df_mock_unchanged)

VIN          1FM5K8GT6HGD50410
Model                 Explorer
ModelYear                 2017
FW:
      ECU                                                            FW  In Database
0  engine     b'GB5A-14C204-PJ\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'        False
1     eps     b'HB53-14D003-AF\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'        False
2     abs  b'HB53-2D053-AB\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'        False

VIN          1FM5K8D8XHGC96884
Model                 Explorer
ModelYear                 2017
FW:
      ECU                                                            FW  In Database
0  engine     b'HB5A-14C204-BC\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'        False
1     eps     b'HB53-14D003-AE\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'        False
2     abs  b'HB53-2D053-AB\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'        False

VIN          1FM5K8D83KGA27066
Model                 Explorer
ModelYear                 2019
FW:
      ECU                  